## Visualizations

This notebook contains visualizations for the gathered data

- Threads x Speedup
- Threads x Efficiency (+ 100% efficiency line)
- Amdahl's law
- Overhead bars
- Solution quality

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
project_root = os.path.abspath('..')
data_path = os.path.join(project_root, 'results', 'data.csv')

In [ ]:
df = pd.read_csv(data_path)
df.head()

In [ ]:
instances = df['instance'].unique()
thread_counts = sorted(df[df['mode'] == 'parallel']['threads'].unique())

In [ ]:
grouped = df.groupby(['instance', 'mode', 'threads'], sort=False).agg({
    'time_ms': 'mean',
    'best_distance': 'mean'
}).reset_index()

sequential_baselines = grouped[grouped['mode'] == 'sequential'].set_index('instance')['time_ms']

grouped['speedup'] = grouped['instance'].map(sequential_baselines) / grouped['time_ms']
grouped['efficiency'] = (grouped['speedup'] / grouped['threads']) * 100

parallel_data = grouped[grouped['mode'] == 'parallel'].copy()
parallel_data['sequential_time'] = parallel_data['instance'].map(sequential_baselines)
parallel_data['ideal_time'] = parallel_data['sequential_time'] / parallel_data['threads']
parallel_data['overhead'] = parallel_data['time_ms'] - parallel_data['ideal_time']

parallel_data.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

for instance in instances:
    instance_data = parallel_data[parallel_data['instance'] == instance]
    
    ax.plot(instance_data['threads'], 
            instance_data['speedup'],
            marker='o', 
            linewidth=2,
            markersize=8,
            label=instance)

ax.set_xlabel('Número de Threads', fontsize=12)
ax.set_ylabel('Speedup', fontsize=12)
ax.set_title('Speedup vs Número de Threads', fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_xticks(thread_counts)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

for instance in instances:
    instance_data = parallel_data[parallel_data['instance'] == instance]
    
    ax.plot(instance_data['threads'], 
            instance_data['efficiency'],
            marker='o', 
            linewidth=2,
            markersize=8,
            label=instance)
    
ax.axhline(y=100, 
        linestyle='--', 
        linewidth=2,
        color='gray', 
        alpha=0.7,
        label='100% Efficiency')

ax.set_xlabel('Número de Threads', fontsize=12)
ax.set_ylabel('Eficiência', fontsize=12)
ax.set_title('Eficiência vs Número de Threads', fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_xticks(thread_counts)
plt.tight_layout()

In [ ]:
for instance in instances:
    instance_data = parallel_data[parallel_data['instance'] == instance]
    
    if len(instance_data) == 0:
        print(f"Skipping {instance}: no parallel data")
        continue
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    threads = instance_data['threads'].values
    ideal = instance_data['ideal_time'].values
    overhead = instance_data['overhead'].values
    
    x_pos = range(len(threads))
    
    # ===== PLOT 1: Stacked Bar Chart =====
    ax1.bar(x_pos, ideal, label='Tempo Teórico', color='steelblue')
    ax1.bar(x_pos, overhead, bottom=ideal, label='Overhead', color='crimson')
    
    ax1.set_xlabel('Número de Threads', fontsize=12)
    ax1.set_ylabel('Tempo de Execução (ms)', fontsize=12)
    ax1.set_title(f'{instance} - Distribuição do Tempo', fontsize=13, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(threads)
    
    # ===== PLOT 2: Overhead Line Graph =====
    ax2.plot(threads, overhead,
             marker='o',
             linewidth=2,
             markersize=8,
             color='crimson',
             label='Overhead')
    
    ax2.set_xlabel('Número de Threads', fontsize=12)
    ax2.set_ylabel('Overhead (ms)', fontsize=12)
    ax2.set_title(f'{instance} - Crescimento do Overhead', fontsize=13, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xticks(threads)
    ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

In [ ]:
for instance in instances:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    instance_data = grouped[grouped['instance'] == instance]
    seq_data = instance_data[instance_data['mode'] == 'sequential']
    par_data = instance_data[instance_data['mode'] == 'parallel']
    
    seq_distance = seq_data['best_distance'].values[0]
    
    threads = par_data['threads'].values
    par_distances = par_data['best_distance'].values
    
    x_pos = range(len(threads))
    
    ax.bar(x_pos, par_distances, color='steelblue', alpha=0.8, label='Paralelo')
    
    ax.axhline(y=seq_distance, color='crimson', linestyle='--', 
               linewidth=2, label='Sequencial')
    
    ax.set_xlabel('Número de Threads', fontsize=12)
    ax.set_ylabel('Distância da Solução', fontsize=12)
    ax.set_title(f'{instance} - Qualidade da Solução', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(threads)
    ax.legend(fontsize=11)
    
    plt.tight_layout()
    plt.show()